In [1]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from huggingface_hub import login
import torch
import os
import json
from PIL import Image
from tqdm import tqdm
import numpy as np
import re
from torch.nn.utils.rnn import pad_sequence

/home/kezouke/Thesis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Login to Hugging Face
your_hf_token = ""
login(token=your_hf_token)

# Configuration
MODEL_PATH = "llava-hf/llama3-llava-next-8b-hf"
IMAGE_DIR = "Thesis/coco2014/test2014"
OUTPUT_FILE = "llama3_llava_next_8b_coco_results_test.json"
BATCH_SIZE = 8
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Generation parameters
GENERATION_PARAMS = {
    "max_new_tokens": 70,
    "do_sample": False,
    "use_cache": True,
}

In [3]:
torch.cuda.empty_cache()
if torch.cuda.is_available():
    print(f"Free GPU memory: {torch.cuda.mem_get_info()[0] / 1024**3:.2f} GB")

Free GPU memory: 23.30 GB


In [4]:
def load_model_and_processor(model_path, device):
    """Load LLaVA-NeXT model and processor."""
    print(f"Loading model: {model_path}")
    
    processor = LlavaNextProcessor.from_pretrained(model_path)
    model = LlavaNextForConditionalGeneration.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=False
    ).eval()
    
    print(f"✓ Model and processor loaded successfully on {next(model.parameters()).device}")
    return model, processor

def extract_image_id(filename):
    """
    Extract image ID from COCO test filename.
    Example: 'COCO_test2014_000000123456.jpg' -> 123456
    """
    match = re.search(r'(\d{12})\.(jpg|jpeg|png)$', filename, re.IGNORECASE)
    if match:
        return int(match.group(1))
    else:
        raise ValueError(f"Could not extract image ID from filename: {filename}")

def clean_caption(caption_text):
    """
    Clean the generated caption by removing conversation template artifacts.
    """
    # Method 1: Split on assistant header (most reliable for LLaMA 3 format)
    if "<|start_header_id|>assistant<|end_header_id|>" in caption_text:
        parts = caption_text.split("<|start_header_id|>assistant<|end_header_id|>")
        if len(parts) > 1:
            assistant_response = parts[-1]
            # Remove any remaining special tokens and clean up
            assistant_response = assistant_response.split("<|eot_id|>")[0].strip()
            return assistant_response
    
    # Method 2: Look for "assistant" keyword followed by the actual response
    if "assistant" in caption_text.lower():
        # Split on "assistant" and take the last part
        parts = re.split(r'assistant\s*', caption_text, flags=re.IGNORECASE)
        if len(parts) > 1:
            response = parts[-1].strip()
            # Remove any remaining prompt artifacts
            if response.startswith("Generate a detailed caption"):
                response = response.split("Generate a detailed caption")[-1].strip()
                if response.startswith("for this image in one sentence."):
                    response = response[len("for this image in one sentence."):].strip()
            return response
    
    # Method 3: Remove the user prompt if it appears at the beginning
    prompt_text = "Generate a detailed caption for this image in one sentence."
    if caption_text.startswith(prompt_text):
        caption_text = caption_text[len(prompt_text):].strip()
    
    # Final cleanup: remove any remaining special tokens or artifacts
    caption_text = caption_text.replace("<|start_header_id|>", "")
    caption_text = caption_text.replace("<|end_header_id|>", "")
    caption_text = caption_text.replace("<|eot_id|>", "")
    caption_text = caption_text.replace("user", "").replace("assistant", "")
    caption_text = re.sub(r'\s+', ' ', caption_text).strip()
    
    return caption_text

def generate_caption_single_optimized(image_path, model, processor, device):
    """Generate caption for a single image using LLaVA-NeXT with memory efficiency."""
    try:
        # Open image directly without verification to save time
        image = Image.open(image_path).convert("RGB")
        
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Generate a detailed caption for this image in one sentence."},
                    {"type": "image"},
                ],
            },
        ]
        
        # Apply chat template
        prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
        
        # Process single image
        inputs = processor(
            images=image,
            text=prompt,
            return_tensors="pt",
            padding=False,
            truncation=False
        ).to(device)
        
        # Convert pixel values to float16 for memory efficiency
        if "pixel_values" in inputs:
            inputs["pixel_values"] = inputs["pixel_values"].to(torch.float16)
        
        # Generate with optimized settings
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=GENERATION_PARAMS["max_new_tokens"],
                do_sample=GENERATION_PARAMS["do_sample"],
                use_cache=GENERATION_PARAMS["use_cache"],
                pad_token_id=processor.tokenizer.pad_token_id,
                eos_token_id=processor.tokenizer.eos_token_id
            )
        
        # Decode with special tokens to properly parse the structure
        full_output = processor.decode(output[0], skip_special_tokens=False)
        
        # Clean and extract the actual caption
        clean_caption_text = clean_caption(full_output)
        
        # Final validation
        if not clean_caption_text or len(clean_caption_text) < 5:
            # Fallback: try decoding without special tokens
            fallback_caption = processor.decode(output[0], skip_special_tokens=True).strip()
            clean_caption_text = clean_caption(fallback_caption)
        
        return clean_caption_text if clean_caption_text else "No caption generated"
    
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return "Error generating caption"
    finally:
        # Clear image from memory
        if 'image' in locals():
            del image

def process_batch_efficient(image_paths, model, processor, device):
    """
    Process a batch of images efficiently using single-image processing.
    """
    captions = []
    for img_path in image_paths:
        caption = generate_caption_single_optimized(img_path, model, processor, device)
        captions.append(caption)
        
        # Clear CUDA cache after each image to prevent memory buildup
        torch.cuda.empty_cache()
    
    return captions

def get_image_files(image_dir):
    """Get sorted list of image files with robust error handling."""
    try:
        all_files = os.listdir(image_dir)
        image_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"\nFound {len(image_files)} images in {image_dir}")
        
        # Sort for reproducibility
        image_files.sort()
        
        return image_files
    except Exception as e:
        print(f"Error reading image directory: {e}")
        return []

In [5]:
# Load model and processor
model, processor = load_model_and_processor(MODEL_PATH, DEVICE)

Loading model: llava-hf/llama3-llava-next-8b-hf


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [02:11<00:00, 32.91s/it]


✓ Model and processor loaded successfully on cuda:0


In [6]:
# Get image files
image_files = get_image_files(IMAGE_DIR)
if not image_files:
    raise ValueError("No image files found in the test directory")

# Test on first few images
print("\nTesting on first batch...")
test_batch_size = min(3, len(image_files))
test_paths = [os.path.join(IMAGE_DIR, image_files[i]) for i in range(test_batch_size)]

print("Sample files to process:")
for path in test_paths:
    print(f"  - {path}")

try:
    test_captions = process_batch_efficient(test_paths, model, processor, DEVICE)
    print("\nCLEAN Sample captions from first batch:")
    for i, (filename, caption) in enumerate(zip(image_files[:test_batch_size], test_captions)):
        img_id = extract_image_id(filename)
        print(f"  {i+1}. Image ID: {img_id}")
        print(f"     File: {filename}")
        print(f"     Caption: {caption}")
except Exception as e:
    print(f"Error during test batch processing: {e}")
    raise



Found 40775 images in /home/kezouke/Thesis/coco2014/test2014

Testing on first batch...
Sample files to process:
  - /home/kezouke/Thesis/coco2014/test2014/COCO_test2014_000000000001.jpg
  - /home/kezouke/Thesis/coco2014/test2014/COCO_test2014_000000000014.jpg
  - /home/kezouke/Thesis/coco2014/test2014/COCO_test2014_000000000016.jpg

CLEAN Sample captions from first batch:
  1. Image ID: 1
     File: COCO_test2014_000000000001.jpg
     Caption: A taxi cab parked next to a black iron fence.
  2. Image ID: 14
     File: COCO_test2014_000000000014.jpg
     Caption: A man riding a skateboard down a street next to a man on a motorcycle.
  3. Image ID: 16
     File: COCO_test2014_000000000016.jpg
     Caption: A baseball player swinging a bat at a ball.


In [7]:
# Full inference on test set with efficient batch processing
results = []
total_batches = (len(image_files) + BATCH_SIZE - 1) // BATCH_SIZE

for batch_idx in tqdm(range(total_batches), desc="Processing batches"):
    start_idx = batch_idx * BATCH_SIZE
    end_idx = min((batch_idx + 1) * BATCH_SIZE, len(image_files))
    
    batch_files = image_files[start_idx:end_idx]
    image_paths_batch = []
    image_ids_batch = []
    
    # Prepare batch data
    for filename in batch_files:
        image_path = os.path.join(IMAGE_DIR, filename)
        try:
            if not os.path.exists(image_path):
                print(f"Warning: File not found: {image_path}")
                continue
            
            img_id = extract_image_id(filename)
            image_paths_batch.append(image_path)
            image_ids_batch.append(img_id)
            
        except Exception as e:
            print(f"Error preparing {filename}: {e}")
            continue
    
    if not image_paths_batch:
        print(f"Warning: Empty batch at index {batch_idx}")
        continue
    
    try:
        # Process batch efficiently
        captions = process_batch_efficient(image_paths_batch, model, processor, DEVICE)
        
        # Append to results
        for img_id, caption in zip(image_ids_batch, captions):
            results.append({
                "image_id": img_id,
                "caption": caption
            })
            
        # Save intermediate results every 5 batches for safety
        if batch_idx > 0 and batch_idx % 5 == 0:
            temp_file = OUTPUT_FILE.replace('.json', f'_temp_batch_{batch_idx}.json')
            with open(temp_file, 'w') as f:
                json.dump(results, f, indent=2)
            print(f"  ✓ Saved intermediate results to {temp_file} ({len(results)} captions so far)")
        
        # Clear memory after each batch
        torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"Error processing batch {batch_idx}: {e}")
        continue

Processing batches:   0%|          | 6/5097 [00:45<10:32:59,  7.46s/it]

  ✓ Saved intermediate results to llama3_llava_next_8b_coco_results_test_temp_batch_5.json (48 captions so far)


Processing batches:   0%|          | 11/5097 [01:22<10:33:32,  7.47s/it]

  ✓ Saved intermediate results to llama3_llava_next_8b_coco_results_test_temp_batch_10.json (88 captions so far)


Processing batches:   0%|          | 16/5097 [01:58<10:23:19,  7.36s/it]

...

Processing batches: 100%|█████████▉| 5091/5097 [10:30:43<00:43,  7.31s/it]

  ✓ Saved intermediate results to llama3_llava_next_8b_coco_results_test_temp_batch_5090.json (40728 captions so far)


Processing batches: 100%|█████████▉| 5096/5097 [10:31:19<00:07,  7.29s/it]

  ✓ Saved intermediate results to llama3_llava_next_8b_coco_results_test_temp_batch_5095.json (40768 captions so far)


Processing batches: 100%|██████████| 5097/5097 [10:31:26<00:00,  7.43s/it]


In [8]:
# Save final results
print(f"\nSaving final results to {OUTPUT_FILE}")
with open(OUTPUT_FILE, 'w') as f:
    json.dump(results, f, indent=2)
print(f"✓ Final results saved successfully. Total predictions: {len(results)}")

# Print summary statistics
if results:
    print("\nSummary:")
    print(f"  Total images processed: {len(results)}")
    print(f"  Total images in directory: {len(image_files)}")
    print(f"  Success rate: {len(results)/len(image_files)*100:.1f}%")
    
    # Calculate average caption length (excluding error messages)
    valid_captions = [r['caption'] for r in results if "Error generating caption" not in r['caption'] and "No caption generated" not in r['caption']]
    if valid_captions:
        avg_length = sum(len(caption) for caption in valid_captions) / len(valid_captions)
        print(f"  Average caption length: {avg_length:.1f} characters")
        print(f"  Valid captions: {len(valid_captions)}")
    
    # Show some sample results
    print("\nSample predictions:")
    for i, result in enumerate(results[:5]):
        print(f"  {i+1}. Image ID: {result['image_id']}")
        print(f"     Caption: {result['caption']}")
else:
    print("Warning: No results were generated. Check for errors above.")

print("\nProcessing complete!")


Saving final results to llama3_llava_next_8b_coco_results_test.json
✓ Final results saved successfully. Total predictions: 40775

Summary:
  Total images processed: 40775
  Total images in directory: 40775
  Success rate: 100.0%
  Average caption length: 55.9 characters
  Valid captions: 40775

Sample predictions:
  1. Image ID: 1
     Caption: A taxi cab parked next to a black iron fence.
  2. Image ID: 14
     Caption: A man riding a skateboard down a street next to a man on a motorcycle.
  3. Image ID: 16
     Caption: A baseball player swinging a bat at a ball.
  4. Image ID: 27
     Caption: A dog sitting in the passenger seat of a car with a bicycle on top.
  5. Image ID: 57
     Caption: A man in a red shirt and shorts playing tennis on a clay court.

Processing complete!
